In [1]:
dw_name="lake1"   # Change this to your DW Name
 
import sempy.fabric as fabric
import struct
import sqlalchemy
import pyodbc
import pandas as pd
from notebookutils import mssparkutils
 
#Function to Return sqlalchemt ODBC Engine, given a connection string and using Integrated AAD Auth to Fabric
def create_engine(connection_string : str):
    token = mssparkutils.credentials.getToken('https://analysis.windows.net/powerbi/api').encode("UTF-16-LE")
    token_struct = struct.pack(f'<I{len(token)}s', len(token), token)
    SQL_COPT_SS_ACCESS_TOKEN = 1256
    return sqlalchemy.create_engine("mssql+pyodbc://", creator=lambda: pyodbc.connect(connection_string, attrs_before={SQL_COPT_SS_ACCESS_TOKEN: token_struct}))

StatementMeta(, 85106cea-88f4-4111-90c3-c5c4ae26b30d, 3, Finished, Available, Finished)

In [2]:
tenant_id=spark.conf.get("trident.tenant.id")
workspace_id=spark.conf.get("trident.workspace.id")
lakehouse_id=spark.conf.get("trident.lakehouse.id")
lakehouse_name=spark.conf.get("trident.lakehouse.name")
sql_end_point= fabric.FabricRestClient().get(f"/v1/workspaces/{workspace_id}/lakehouses/{lakehouse_id}").json()['properties']['sqlEndpointProperties']['connectionString']
connection_string = f"Driver={{ODBC Driver 18 for SQL Server}};Server={sql_end_point}"
print (f"connection_string={connection_string}")

StatementMeta(, 85106cea-88f4-4111-90c3-c5c4ae26b30d, 4, Finished, Available, Finished)

connection_string=Driver={ODBC Driver 18 for SQL Server};Server=sre5mxgamrmuzpfgqakspmnfba-qof5ppqwvtoevjs2ev6jxgnd6y.datawarehouse.fabric.microsoft.com


In [4]:
sql_df = pd.read_sql( 
    "SELECT * FROM sys.objects", 
    con=engine 
) 

StatementMeta(, 85106cea-88f4-4111-90c3-c5c4ae26b30d, 6, Finished, Available, Finished)

In [5]:
sql_df

StatementMeta(, 85106cea-88f4-4111-90c3-c5c4ae26b30d, 7, Finished, Available, Finished)

,name,object_id,principal_id,schema_id,parent_object_id,type,type_desc,create_date,modify_date,is_ms_shipped,is_published,is_schema_published
0,sysrscols,3,None,4,0,S,SYSTEM_TABLE,2024-10-03 16:10:26.527,2024-10-03 16:10:27.503,True,False,False
1,sysrowsets,5,None,4,0,S,SYSTEM_TABLE,2009-04-13 12:59:11.093,2024-10-03 16:10:27.030,True,False,False
2,sysclones,6,None,4,0,S,SYSTEM_TABLE,2024-10-03 16:10:26.903,2024-10-03 16:10:26.910,True,False,False
3,sysallocunits,7,None,4,0,S,SYSTEM_TABLE,2009-04-13 12:59:11.077,2024-10-03 16:10:26.553,True,False,False
4,sysfiles1,8,None,4,0,S,SYSTEM_TABLE,2003-04-08 09:13:38.093,2003-04-08 09:13:38.093,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
133,ServiceBrokerQueue,2041058307,None,1,0,SQ,SERVICE_QUEUE,2009-04-13 12:59:13.500,2009-04-13 12:59:13.500,True,False,False
134,queue_messages_2041058307,2057058364,None,4,2041058307,IT,INTERNAL_TABLE,2009-04-13 12:59:13.500,2024-10-03 16:10:26.880,True,False,False
135,filestream_tombstone_2073058421,2073058421,None,4,0,IT,INTERNAL_TABLE,2024-10-03 16:10:26.653,2024-10-03 16:10:26.927,True,False,False
136,syscommittab,2089058478,None,4,0,IT,INTERNAL_TABLE,2024-10-03 16:10:26.687,2024-10-03 16:10:26.687,True,False,False


In [3]:
engine = create_engine(connection_string)
with engine.connect() as alchemy_connection:
    #Run TSQL Query on a LH End Point
    query = f"exec {lakehouse_name}.[sys].[sp_server_info] 2"
    df = pd.read_sql_query(query, alchemy_connection)
    print (df)
 
    #Run TSQL Query on a DW End Point
    query = f"exec {dw_name}.[sys].[sp_server_info] 2"
    df = pd.read_sql_query(query, alchemy_connection)
    print (df)
 
    #Execute a TSQL Stored Procedure or DDL/DML on a Fabric DW
    connection = engine.raw_connection()
    cursor = connection.cursor()
    sql= f"USE {dw_name};CREATE TABLE tmpTable (Column1 INT NULL);DROP TABLE tmpTable"
    cursor.execute(sql)
    connection.commit()

StatementMeta(, 85106cea-88f4-4111-90c3-c5c4ae26b30d, 5, Finished, Available, Finished)

   attribute_id attribute_name                            attribute_value
0             2       DBMS_VER  Microsoft SQL Server 2022 - 16.0.5749.110
   attribute_id attribute_name                            attribute_value
0             2       DBMS_VER  Microsoft SQL Server 2022 - 16.0.5749.110
